In [21]:

class Scalar1:

    def __init__(self, v):
        self.v = v
        self.grad = 0        

    def __repr__(self):
        return f"Scalar(v={self.v}, grad={self.grad})"

    def __add__(self, o):
        self.grad += 1
        o.grad += 1
        return Scalar(self.v + o.v)

    def __mul__(self, o):
        self.grad += o.v
        o.grad += self.v
        return Scalar(self.v * o.v)

a = Scalar1(2)
b = Scalar1(3)
c = Scalar1(4)
z = a + b * c
z

Scalar(v=14, grad=0)

In [20]:

class Scalar:

    def __init__(self, v, children=tuple(), back=lambda: None, op=""):
        self.v = v
        self.children = children
        self.back = back
        self.grad = 0
        self.op = op

    def __repr__(self):
        return f"Scalar(v={self.v}, grad={self.grad})"

    def __add__(self, o):
        def __back():
            self.grad += 1
            o.grad += 1
        return Scalar(self.v + o.v, children=(self, o), back=__back, op="+")

    def __mul__(self, o):
        def __back():
            self.grad += o.v
            o.grad += self.v
        return Scalar(self.v * o.v, children=(self, o), back=__back, op="*")

    def backprop(self):
        from collections import deque
        children = deque()
        children.append(self)
        while children:
            c = children.popleft()
            c.back()
            children.extend(c.children)
    

    def backprop_1(self):
        self.back()
        for c in self.children:
            c.back()

a = Scalar(1)
b = Scalar(2)
c = Scalar(3)
d = b * c
z = a + d
z.backprop()
print(f"a={a}, b={b}, c={c}, d={d}, z={z}")

eps = 0.00001
grad_a = (((a.v + eps) + b.v * c.v) - (a.v + b.v * c.v)) / eps
grad_b = ((a.v + (b.v + eps) * c.v) - (a.v + b.v * c.v)) / eps
grad_c = ((a.v + b.v * (c.v + eps)) - (a.v + b.v * c.v)) / eps
print(grad_a, grad_b, grad_c)

a = Scalar(1)
b = Scalar(2)
c = Scalar(3)
d = Scalar(4)

e = c + d
f = b * e
z = a + f
z.backprop()
print(f"a={a}, b={b}, c={c}, d={d}, e={e}, f={f}, z={z}")

eps = 0.00001
grad_a = (((a.v + eps) + b.v * (c.v + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_b = ((a.v + (b.v + eps) * (c.v + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_c = ((a.v + b.v * (c.v + eps + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_d = ((a.v + b.v * (c.v + d.v + eps)) - (a.v + b.v * (c.v + d.v))) / eps
print(grad_a, grad_b, grad_c, grad_d)

a=Scalar(v=1, grad=1), b=Scalar(v=2, grad=3), c=Scalar(v=3, grad=2), d=Scalar(v=6, grad=1), z=Scalar(v=7, grad=0)
0.9999999999621422 3.000000000064062 2.0000000000131024
a=Scalar(v=1, grad=1), b=Scalar(v=2, grad=7), c=Scalar(v=3, grad=1), d=Scalar(v=4, grad=1), e=Scalar(v=7, grad=2), f=Scalar(v=14, grad=1), z=Scalar(v=15, grad=0)
0.9999999999621422 7.0000000000902665 1.9999999999242843 1.9999999999242843


In [29]:
class Scalar:

    def __init__(self, v, children=tuple(), back=lambda: None, op=""):
        self.v = v
        self.children = children
        self.back = back
        self.grad = 1
        self.op = op

    def __repr__(self):
        return f"Scalar(v={self.v}, grad={self.grad})"

    def __add__(self, o):
        def __back():
            for c in self.children:
                c.grad = self.grad
            for c in self.children:
                c.grad = o.grad
        return Scalar(self.v + o.v, children=(self, o), back=__back, op="+")

    def __mul__(self, o):
        def __back():
            self.grad *= o.v
            for c in self.children:
                c.grad = self.grad
            o.grad *= self.v
            for c in o.children:
                c.grad = self.v
        return Scalar(self.v * o.v, children=(self, o), back=__back, op="*")

    def backprop(self):
        from collections import deque
        children = deque()
        children.append(self)
        while children:
            c = children.popleft()
            c.back()
            children.extend(c.children)
    

    def backprop_1(self):
        self.back()
        for c in self.children:
            c.back()

a = Scalar(1)
b = Scalar(2)
c = Scalar(3)
d = b * c
z = a + d
z.backprop()
print(f"a={a}, b={b}, c={c}, d={d}, z={z}")

eps = 0.00001
grad_a = (((a.v + eps) + b.v * c.v) - (a.v + b.v * c.v)) / eps
grad_b = ((a.v + (b.v + eps) * c.v) - (a.v + b.v * c.v)) / eps
grad_c = ((a.v + b.v * (c.v + eps)) - (a.v + b.v * c.v)) / eps
print(grad_a, grad_b, grad_c)

a = Scalar(1)
b = Scalar(2)
c = Scalar(3)
d = Scalar(4)

e = c + d
f = b * e
z = a + f
z.backprop()
print(f"a={a}, b={b}, c={c}, d={d}, e={e}, f={f}, z={z}")

eps = 0.00001
grad_a = (((a.v + eps) + b.v * (c.v + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_b = ((a.v + (b.v + eps) * (c.v + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_c = ((a.v + b.v * (c.v + eps + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_d = ((a.v + b.v * (c.v + d.v + eps)) - (a.v + b.v * (c.v + d.v))) / eps
print(grad_a, grad_b, grad_c, grad_d)


a = Scalar(1)
b = Scalar(2)
c = Scalar(3)
d = Scalar(4)
e = Scalar(5)

f = d * e
g = c + f
h = b * g
z = a + h
z.backprop()
print(f"a={a}, b={b}, c={c}, d={d}, e={e}, f={f}, g={g}, h={h}, z={z}")

eps = 0.00001
grad_a = (((a.v + eps) + b.v * (c.v + d.v * e.v)) - (a.v + b.v * (c.v + d.v * e.v))) / eps
grad_b = ((a.v + (b.v + eps) * (c.v + d.v * e.v)) - (a.v + b.v * (c.v + d.v * e.v))) / eps
grad_c = ((a.v + b.v * (c.v + eps + d.v * e.v)) - (a.v + b.v * (c.v + d.v * e.v))) / eps
grad_d = ((a.v + b.v * (c.v + (d.v + eps) * e.v )) - (a.v + b.v * (c.v + d.v * e.v))) / eps
grad_e = ((a.v + b.v * (c.v +  d.v  * (e.v + eps))) - (a.v + b.v * (c.v + d.v * e.v))) / eps
print(grad_a, grad_b, grad_c, grad_d, grad_e)

a=Scalar(v=1, grad=1), b=Scalar(v=2, grad=3), c=Scalar(v=3, grad=2), d=Scalar(v=6, grad=1), z=Scalar(v=7, grad=1)
0.9999999999621422 3.000000000064062 2.0000000000131024
a=Scalar(v=1, grad=1), b=Scalar(v=2, grad=7), c=Scalar(v=3, grad=2), d=Scalar(v=4, grad=2), e=Scalar(v=7, grad=2), f=Scalar(v=14, grad=1), z=Scalar(v=15, grad=1)
0.9999999999621422 7.0000000000902665 1.9999999999242843 1.9999999999242843
a=Scalar(v=1, grad=1), b=Scalar(v=2, grad=23), c=Scalar(v=3, grad=2), d=Scalar(v=4, grad=5), e=Scalar(v=5, grad=4), f=Scalar(v=20, grad=2), g=Scalar(v=23, grad=2), h=Scalar(v=46, grad=1), z=Scalar(v=47, grad=1)
1.0000000003174137 23.000000000195083 1.9999999999242843 9.999999999621423 7.999999999697137


In [30]:
class Scalar:

    def __init__(self, v, children=tuple(), back=lambda: None, op=""):
        self.v = v
        self.children = children
        self.back = back
        self.grad = 1
        self.op = op

    def __repr__(self):
        return f"Scalar(v={self.v}, grad={self.grad})"

    def __add__(self, o):
        s = Scalar(self.v + o.v, children=(self, o), op="+")
        def __back():
            self.grad += s.grad
            o.grad += s.grad
        s.back = __back
        return s

    def __mul__(self, o):
        s = Scalar(self.v * o.v, children=(self, o), op="*")
        def __back():
            self.grad += s.grad * o.v
            o.grad += s.grad * self.v
        s.back = __back
        return s

    def backprop(self):
        from collections import deque
        children = deque()
        children.append(self)
        while children:
            c = children.popleft()
            c.back()
            children.extend(c.children)
    

    def backprop_1(self):
        self.back()
        for c in self.children:
            c.back()

a = Scalar(1)
b = Scalar(2)
c = Scalar(3)
d = b * c
z = a + d
z.backprop()
print(f"a={a}, b={b}, c={c}, d={d}, z={z}")

eps = 0.00001
grad_a = (((a.v + eps) + b.v * c.v) - (a.v + b.v * c.v)) / eps
grad_b = ((a.v + (b.v + eps) * c.v) - (a.v + b.v * c.v)) / eps
grad_c = ((a.v + b.v * (c.v + eps)) - (a.v + b.v * c.v)) / eps
print(grad_a, grad_b, grad_c)

a = Scalar(1)
b = Scalar(2)
c = Scalar(3)
d = Scalar(4)

e = c + d
f = b * e
z = a + f
z.backprop()
print(f"a={a}, b={b}, c={c}, d={d}, e={e}, f={f}, z={z}")

eps = 0.00001
grad_a = (((a.v + eps) + b.v * (c.v + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_b = ((a.v + (b.v + eps) * (c.v + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_c = ((a.v + b.v * (c.v + eps + d.v)) - (a.v + b.v * (c.v + d.v))) / eps
grad_d = ((a.v + b.v * (c.v + d.v + eps)) - (a.v + b.v * (c.v + d.v))) / eps
print(grad_a, grad_b, grad_c, grad_d)


a = Scalar(1)
b = Scalar(2)
c = Scalar(3)
d = Scalar(4)
e = Scalar(5)

f = d * e
g = c + f
h = b * g
z = a + h
z.backprop()
print(f"a={a}, b={b}, c={c}, d={d}, e={e}, f={f}, g={g}, h={h}, z={z}")

eps = 0.00001
grad_a = (((a.v + eps) + b.v * (c.v + d.v * e.v)) - (a.v + b.v * (c.v + d.v * e.v))) / eps
grad_b = ((a.v + (b.v + eps) * (c.v + d.v * e.v)) - (a.v + b.v * (c.v + d.v * e.v))) / eps
grad_c = ((a.v + b.v * (c.v + eps + d.v * e.v)) - (a.v + b.v * (c.v + d.v * e.v))) / eps
grad_d = ((a.v + b.v * (c.v + (d.v + eps) * e.v )) - (a.v + b.v * (c.v + d.v * e.v))) / eps
grad_e = ((a.v + b.v * (c.v +  d.v  * (e.v + eps))) - (a.v + b.v * (c.v + d.v * e.v))) / eps
print(grad_a, grad_b, grad_c, grad_d, grad_e)

a=Scalar(v=1, grad=2), b=Scalar(v=2, grad=7), c=Scalar(v=3, grad=5), d=Scalar(v=6, grad=2), z=Scalar(v=7, grad=1)
0.9999999999621422 3.000000000064062 2.0000000000131024
a=Scalar(v=1, grad=2), b=Scalar(v=2, grad=15), c=Scalar(v=3, grad=6), d=Scalar(v=4, grad=6), e=Scalar(v=7, grad=5), f=Scalar(v=14, grad=2), z=Scalar(v=15, grad=1)
0.9999999999621422 7.0000000000902665 1.9999999999242843 1.9999999999242843
a=Scalar(v=1, grad=2), b=Scalar(v=2, grad=47), c=Scalar(v=3, grad=6), d=Scalar(v=4, grad=31), e=Scalar(v=5, grad=25), f=Scalar(v=20, grad=6), g=Scalar(v=23, grad=5), h=Scalar(v=46, grad=2), z=Scalar(v=47, grad=1)
1.0000000003174137 23.000000000195083 1.9999999999242843 9.999999999621423 7.999999999697137
